In [1]:
import os
import calendar
import datetime
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm

from PIL import Image
from PIL.ExifTags import TAGS

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/2p/xkb_dxsd3sqfntxcfrr_n0lc0000gn/T/ipykernel_6778/1000898102.py:15: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
df_test = pd.read_csv('df_auto.csv', sep=",")

In [5]:
df = pd.DataFrame(columns=['locations', "start_date (YYYY-MM-DD)", "end_date (YYYY-MM-DD)"])

In [18]:
df['locations'] = ["Cambodia"]
df['start_date (YYYY-MM-DD)'] = pd.to_datetime(['2021-07-04'])
df['end_date (YYYY-MM-DD)'] = pd.to_datetime(['2021-08-04'])

In [19]:
df_test.head()

,Unnamed: 0,from_path,filetype,date,manual_sort,photo_year,photo_month,photo_month_name,to_path
0,0,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/IMG_20210113_093540.jpg,photo,2021-01-13 09:35:44,False,2021,1,01_Jan,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/01_Jan
1,1,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05084.JPG,photo,2021-07-05 10:59:33,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul
2,2,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05078.JPG,photo,2021-07-05 10:58:12,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul
3,3,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05337.JPG,photo,2012-09-08 19:17:59,False,2012,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2012/09_Sep
4,4,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05080.JPG,photo,2021-07-05 10:58:14,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul


In [20]:
df.head()

,locations,start_date (YYYY-MM-DD),end_date (YYYY-MM-DD)
0,Cambodia,2021-07-04,2021-08-04


In [39]:
def extract_location(date, df_locations):
    
    df_time_window = df[(df['start_date (YYYY-MM-DD)'] < date) & (df['end_date (YYYY-MM-DD)'] > date)].reset_index()
    
    if len(df_time_window) > 1:
        print('Several dates are overlapping')
    elif len(df_time_window) == 0:
        return np.nan
    
    return df_time_window['locations'].item()
    

In [44]:
def add_location_to_path(path, location):
    if not pd.isna(location):
        new_path = '{}_{}'.format(path, location)
        return new_path
    return path

In [45]:
df_test['location'] = df_test['date'].map(lambda x: extract_location(x, df))
df_test['to_path'] = df_test.apply(lambda x: add_location_to_path(x['to_path'], x['location']), axis=1)

        

In [46]:
df_test

,Unnamed: 0,from_path,filetype,date,manual_sort,photo_year,photo_month,photo_month_name,to_path,location
0,0,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/IMG_20210113_093540.jpg,photo,2021-01-13 09:35:44,False,2021,1,01_Jan,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/01_Jan,NaN
1,1,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05084.JPG,photo,2021-07-05 10:59:33,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul_Cambodia,Cambodia
2,2,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05078.JPG,photo,2021-07-05 10:58:12,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul_Cambodia,Cambodia
3,3,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05337.JPG,photo,2012-09-08 19:17:59,False,2012,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2012/09_Sep,NaN
4,4,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/DSC05080.JPG,photo,2021-07-05 10:58:14,False,2021,7,07_Jul,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2021/07_Jul_Cambodia,Cambodia
...,...,...,...,...,...,...,...,...,...,...
57,57,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/sous_dossier_1/Photos & vidéos/photo shoot/DSC_1331.jpg,photo,2019-09-14 17:08:02,False,2019,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2019/09_Sep,NaN
58,58,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/sous_dossier_1/Photos & vidéos/photo shoot/DSC_1325.jpg,photo,2019-09-14 17:06:51,False,2019,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2019/09_Sep,NaN
59,59,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/sous_dossier_1/Photos & vidéos/photo shoot/DSC_1323.jpg,photo,2019-09-14 17:05:34,False,2019,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2019/09_Sep,NaN
60,60,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/fileBucket/sous_dossier_1/Photos & vidéos/photo shoot/DSC_1337.jpg,photo,2019-09-14 17:08:38,False,2019,9,09_Sep,/Users/raphaelmatusiak/Documents/Perso/projet perso/sortphoto/sorted_photos/2019/09_Sep,NaN


In [ ]:
df.to_csv('locations.csv', sep=";", index=False)